# Installations

In [2]:
!pip install rasterio

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 9.9/21.5 MB 112.7 kB/s eta 0:01:44^C4
   ━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 9.9/21.5 MB 112.7 kB/s eta 0:01:44
ERROR: Operation cancelled by user


# Image Visualization

### Original image

In [1]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# File path to the TIFF image
file_path = r'D:\Bachelors_Degree_Work\Sem_7\Data_Vis_Lab\DL\original.tif'

# Open the TIFF file
with rasterio.open(file_path) as src: 
    # Check the number of layers in the TIFF file
    num_layers = src.count
    
    # Loop through each layer and display it
    for i in range(1, num_layers + 1):
        plt.figure(figsize=(10, 10))
        show((src, i), title=f"Layer {i}", cmap='grey')
        plt.show()


ModuleNotFoundError: No module named 'rasterio'

### Classified Image

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

# File path to the TIFF image
file_path = r'D:\Bachelors_Degree_Work\Sem_7\Data_Vis_Lab\DL\classified.tif'

# Open the TIFF file
with rasterio.open(file_path) as src:
    # Check the number of layers in the TIFF file
    numbers = src.read()
    print(np.unique(np.array(numbers)))
    num_layers = src.count
    
    # Loop through each layer and display it
    for i in range(1, num_layers + 1):
        plt.figure(figsize=(10, 10))
        show((src, i), title=f"Layer {i}", cmap='grey')
        plt.show()


# Data Labelling

In [ ]:
import rasterio
from rasterio.plot import show
import numpy as np

# Paths to the original image and classified image
original_image_path = 'D:/Bachelors_Degree_Work/Sem_7/Data_Vis_Lab/DL/original.tif'
classified_image_path = 'D:/Bachelors_Degree_Work/Sem_7/Data_Vis_Lab/DL/CLASSIFIED.tif'

# Open the original image using Rasterio
with rasterio.open(original_image_path) as original_image_ds:
    # Read the spectral bands from the original image
    stacked_image = original_image_ds.read()

# Open the classified image using Rasterio
with rasterio.open(classified_image_path) as classified_image_ds:
    # Read the classified image
    classified_image = classified_image_ds.read(1)  # Assuming the classified image has a single band

# Get the unique classes present in the classified image
classes = np.unique(classified_image)

# Initialize dictionaries to store spectral information for each class
class_spectral_info = {c: [] for c in classes}

# Iterate over each pixel in the classified image
for i in range(classified_image.shape[0]):
    for j in range(classified_image.shape[1]):
        # Get the class label for the current pixel
        class_label = classified_image[i, j]
        
        # Get the spectral information for the current pixel from the stacked satellite image
        spectral_info = stacked_image[:, i, j]
        
        # Append the spectral information to the corresponding class array
        class_spectral_info[class_label].append(spectral_info)

# Convert lists to numpy arrays
for c in class_spectral_info:
    class_spectral_info[c] = np.array(class_spectral_info[c])

# Now, class_spectral_info dictionary contains arrays of spectral information for each class


In [ ]:
# Print the spectral information for each class
for c in class_spectral_info:
    print(f"Class {c}:")
    print(class_spectral_info[c])
    print("Number of pixels:", len(class_spectral_info[c]))
    print()


### Signatures of each class

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the spectral profile for the first 5 pixels of each class
for c in class_spectral_info:
    # Extract spectral information for the first 5 pixels of the class
    spectral_info = class_spectral_info[c][100:150]

    # Transpose the spectral_info array to have bands on the x-axis and pixel values on the y-axis
    spectral_info = spectral_info.T

    # Plot the spectral profile
    plt.figure(figsize=(15, 6))
    sns.lineplot(data=spectral_info, dashes=False)
    plt.title(f"Spectral Profile for Class {c}")
    plt.xlabel("Band Number")
    plt.ylabel("Pixel Value")
    plt.show()


# CNN Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Print the spectral information for each class
for c in class_spectral_info:
    if c == 0:
        print("Class Snow:")
    elif c == 1:
        print("Class Clouds:")
    elif c == 2:
        print("Class NonSnow:")
    print(class_spectral_info[c])
    print("Number of pixels:", len(class_spectral_info[c]))
    print()

# Concatenate spectral information for all classes into a single array
X = np.concatenate([class_spectral_info[c] for c in class_spectral_info])

# Create corresponding labels for the spectral information
y = np.concatenate([np.full_like(class_spectral_info[c][:, 0], c) for c in class_spectral_info])

# Shuffle the data
perm = np.random.permutation(len(X))
X = X[perm]
y = y[perm]

# Normalize the input data
X_normalized = X / np.max(X)

# Define the CNN architecture
model = models.Sequential([
    layers.Reshape((X.shape[1], 1), input_shape=(X.shape[1],)),
    layers.Conv1D(32, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_spectral_info), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_normalized, y, epochs=30, batch_size=32, validation_split=0.2)

# Save the trained model
model.save('spectral_classification_model2.h5')


### Results

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')

plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(model.history.history['accuracy'], label='Training Accuracy')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(X_normalized, y, verbose=0)
print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_normalized, y, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Calculate other statistics if needed


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predict class labels for the data
y_pred = np.argmax(model.predict(X_normalized), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y, y_pred, average='macro')
recall = recall_score(y, y_pred, average='macro')
f1 = f1_score(y, y_pred, average='macro')

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


# Data Cleaning

In [ ]:
import rasterio
import numpy as np
from sklearn.ensemble import IsolationForest

# Paths to the original image and classified image
original_image_path = 'D:/Bachelors_Degree_Work/Sem_7/Data_Vis_Lab/DL/original.tif'
classified_image_path = 'D:/Bachelors_Degree_Work/Sem_7/Data_Vis_Lab/DL/CLASSIFIED.tif'

# Open the original image using Rasterio
with rasterio.open(original_image_path) as original_image_ds:
    # Read the spectral bands from the original image
    stacked_image = original_image_ds.read()

# Open the classified image using Rasterio
with rasterio.open(classified_image_path) as classified_image_ds:
    # Read the classified image
    classified_image = classified_image_ds.read(1)  # Assuming the classified image has a single band

# Get the unique classes present in the classified image
classes = np.unique(classified_image)

# Initialize dictionaries to store spectral information for each class
class_spectral_info = {c: [] for c in classes}

# Iterate over each pixel in the classified image
for i in range(classified_image.shape[0]):
    for j in range(classified_image.shape[1]):
        # Get the class label for the current pixel
        class_label = classified_image[i, j]
        
        # Get the spectral information for the current pixel from the stacked satellite image
        spectral_info = stacked_image[:, i, j]
        
        # Append the spectral information to the corresponding class array
        class_spectral_info[class_label].append(spectral_info)

# Convert lists to numpy arrays
for c in class_spectral_info:
    class_spectral_info[c] = np.array(class_spectral_info[c])

# Apply Isolation Forest to each class's spectral data to identify and remove anomalies
cleaned_class_spectral_info = {}

for c in class_spectral_info:
    # Fit the Isolation Forest model
    iso_forest = IsolationForest(contamination='auto')  # 'auto' uses a threshold that automatically determines the fraction of outliers
    preds = iso_forest.fit_predict(class_spectral_info[c])
    
    # Filter out the anomalies (outliers are marked as -1)
    filtered_spectral_info = class_spectral_info[c][preds == 1]
    
    # Update the cleaned spectral information dictionary
    cleaned_class_spectral_info[c] = filtered_spectral_info
    
    print(f"Class {c}: Original {len(class_spectral_info[c])}, Cleaned {len(filtered_spectral_info)}")

# Now, cleaned_class_spectral_info dictionary contains arrays of spectral information for each class, with anomalies removed.


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Assuming `cleaned_class_spectral_info` is your final preprocessed data
# Convert the dictionary to a dataset suitable for training
# Flatten the spatial dimensions if your data includes them, or directly use the spectral data as features

X = []
y = []

# Loop through each class in the cleaned data
for class_label, spectral_data in cleaned_class_spectral_info.items():
    for pixel_data in spectral_data:
        X.append(pixel_data)  # Append the spectral data
        y.append(class_label)  # Append the class label

# Convert lists to numpy arrays for TensorFlow compatibility
X = np.array(X)
y = np.array(y)

# Normalize X to be in the range [0, 1]
X = X / X.max()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert class vectors to binary class matrices (for use with categorical_crossentropy)
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

# Define the CNN model
model = tf.keras.Sequential([
    # Add a reshape layer to match the input shape required by the CNN
    tf.keras.layers.Reshape((12, 1, 1), input_shape=(12,)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


In [ ]:
import matplotlib.pyplot as plt

# Define bands to plot (e.g., Band 3 vs. Band 4). Adjust these indices based on your interest.
band_x_index = 9  # Assuming 0-based indexing, this could be the 'Green' band for many satellite images.
band_y_index = 11  # This could be the 'Red' band, for instance.

# Plotting original vs. cleaned spectral data for each class
for c in cleaned_class_spectral_info:
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
    fig.suptitle(f'Class {c} Spectral Data: Original vs. Cleaned')
    
    # Original spectral data plot
    axs[0].scatter(class_spectral_info[c][:, band_x_index], class_spectral_info[c][:, band_y_index], alpha=0.5)
    axs[0].set_title('Original')
    axs[0].set_xlabel(f'Band {band_x_index + 1}')
    axs[0].set_ylabel(f'Band {band_y_index + 1}')
    
    # Cleaned spectral data plot
    axs[1].scatter(cleaned_class_spectral_info[c][:, band_x_index], cleaned_class_spectral_info[c][:, band_y_index], alpha=0.5)
    axs[1].set_title('Cleaned')
    axs[1].set_xlabel(f'Band {band_x_index + 1}')
    # axs[1].set_ylabel(f'Band {band_y_index + 1}')  # No need, shared y-axis
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Function to plot mean spectral signature with standard deviation
def plot_spectral_signature(class_spectral_data, class_label, title):
    # Calculate mean and standard deviation across pixels for each band
    mean_spectral = np.mean(class_spectral_data, axis=0)
    std_spectral = np.std(class_spectral_data, axis=0)
    bands = range(1, class_spectral_data.shape[1] + 1)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.title(f'{title} - Class {class_label}')
    plt.plot(bands, mean_spectral, label='Mean Spectral Signature')
    plt.fill_between(bands, mean_spectral - std_spectral, mean_spectral + std_spectral, alpha=0.2)
    plt.xlabel('Band Number')
    plt.ylabel('Reflectance Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plotting for each class - Original and Cleaned
for c in classes:
    plot_spectral_signature(class_spectral_info[c], c, 'Original Spectral Signature')
    plot_spectral_signature(cleaned_class_spectral_info[c], c, 'Cleaned Spectral Signature')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_pixel_spectra(class_spectral_data, class_label, title, sample_size=10000):
    """
    Plot the spectral data for a sample of pixels in a class.
    
    Parameters:
    - class_spectral_data: numpy array of shape (n_pixels, n_bands)
    - class_label: label for the class
    - title: title for the plot
    - sample_size: number of pixels to randomly sample for plotting
    """
    # Ensure sample size is not larger than the number of pixels in the class
    sample_size = min(sample_size, class_spectral_data.shape[0])
    
    # Randomly sample pixels to reduce plot clutter
    if sample_size < class_spectral_data.shape[0]:
        indices = np.random.choice(class_spectral_data.shape[0], sample_size, replace=False)
        sampled_data = class_spectral_data[indices, :]
    else:
        sampled_data = class_spectral_data
    
    # Plot each sampled pixel's spectral data
    plt.figure(figsize=(10, 6))
    for i in range(sampled_data.shape[0]):
        plt.plot(range(1, sampled_data.shape[1] + 1), sampled_data[i, :], lw=1, alpha=0.5)
    
    plt.title(f'{title} - Class {class_label}')
    plt.xlabel('Band Number')
    plt.ylabel('Reflectance Value')
    plt.grid(True)
    plt.show()

# Number of pixels to sample for each plot
sample_size = 10000  # Adjust based on your dataset size and diversity

# Plotting for each class - Original and Cleaned
for c in classes:
    plot_pixel_spectra(class_spectral_info[c], c, 'Original Pixel Spectra', sample_size)
    plot_pixel_spectra(cleaned_class_spectral_info[c], c, 'Cleaned Pixel Spectra', sample_size)
